In [5]:
import tensorflow as tf
import numpy as np
from emnist import extract_training_samples, extract_test_samples
x_train, y_train = extract_training_samples('letters')
x_test, y_test = extract_test_samples('letters')
x_train

array([[[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       ...,

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 

In [6]:
y_train = np.array(y_train)
t_test = np.array(y_test)
y_train = y_train - 1
y_test = y_test - 1
np.unique(y_train)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25], dtype=uint8)

In [7]:
#make pixel values from 0 to 1

x_train = x_train/255.0
x_test = x_test/255.0

In [8]:
model = tf.keras.Sequential([tf.keras.layers.Flatten(input_shape = (28,28))
                            , tf.keras.layers.Dense(128, activation = 'relu')
                            , tf.keras.layers.Dense(128, activation = 'relu')
                            , tf.keras.layers.Dense(26)])

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [9]:
#compile

model.compile(optimizer = 'adam',
              loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True),
              metrics = ['accuracy'])

In [10]:
#start training

model.fit(x_train,y_train, epochs = 10)

Epoch 1/10
124800/124800 [==============================] - 12s 96us/sample - loss: 0.6560 - acc: 0.8013
Epoch 2/10
124800/124800 [==============================] - 11s 88us/sample - loss: 0.3747 - acc: 0.8802
Epoch 3/10
124800/124800 [==============================] - 11s 88us/sample - loss: 0.3116 - acc: 0.8983
Epoch 4/10
124800/124800 [==============================] - 12s 96us/sample - loss: 0.2761 - acc: 0.9082
Epoch 5/10
124800/124800 [==============================] - 11s 88us/sample - loss: 0.2506 - acc: 0.9149
Epoch 6/10
124800/124800 [==============================] - 11s 88us/sample - loss: 0.2290 - acc: 0.9223
Epoch 7/10
124800/124800 [==============================] - 11s 90us/sample - loss: 0.2162 - acc: 0.9250
Epoch 8/10
124800/124800 [==============================] - 11s 90us/sample - loss: 0.2045 - acc: 0.9277
Epoch 9/10
124800/124800 [==============================] - 11s 91us/sample - loss: 0.1924 - acc: 0.9317
Epoch 10/10
124800/124800 [============================

In [11]:
#import things

import cv2
#import numpy  as np
from collections import deque
import time
import pyttsx3


In [12]:
prediction_list = ["a", "b", "c", "d", "e", "f", "g", "h", "i", "j", "k", "l", "m", "n", "o", "p", "q", "r",
                  "s", "t", "u", "v", "w", "x", "y", "z"]

In [13]:
#define things
#engine = pyttsx3.init()
points = deque(maxlen = 1000)
red_upper = (150,150,50)
red_lower = (200,255,150)
blackboard = np.zeros((512,512), np.uint8)

In [21]:
start_time = time.time()


cap = cv2.VideoCapture(0)
points
    
while time.time() < start_time + 20:
    _, frame = cap.read()
    frame = cv2.flip(frame, 1)
    #hsv
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    #red
    red = cv2.inRange(hsv, red_upper, red_lower)
    #red median blur
    red_median = cv2.medianBlur(red, 15)
    #find contours
    _, contours, _ = cv2.findContours(red_median, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    try:
        if len(contours) > 0:
            contour_list = sorted(contours, key = cv2.contourArea, reverse = True)
            contour = contour_list[0]
            #find centroid
            M = cv2.moments(contour) #M will be a dict
            centroid_x = int(M["m10"]/M["m00"])
            centroid_y = int(M["m01"]/M["m00"])
            centroid = (centroid_x, centroid_y)
            #append centroid in points
            points.appendleft(centroid)
            #draw circle
            ((x,y), r) = cv2.minEnclosingCircle(contour)
            cv2.circle(frame, (int(x),int(y)), int(r), (0,255,0), 2)
    except ZeroDivisionError:
        pass
            
    for i in range(1, len(points)):
        cv2.line(frame, tuple(map(int,points[i])), tuple(map(int,points[i-1])), (0,0,0), 2)
        cv2.line(blackboard, tuple(map(int,points[i])), tuple(map(int,points[i-1])), (255,255,255), 8)

    cv2.imshow("frame", frame)
    cv2.imshow("blackboard", blackboard)
    
    if cv2.waitKey(1) &  0xFF == ord("q"):
        break

    
blackboard_median = cv2.medianBlur(blackboard, 15)
alphabet = cv2.resize(blackboard_median, (28,28))
alphabet = np.array(alphabet)
alphabet = alphabet.astype("float32")/255
prediction = model.predict(alphabet.reshape(1,28,28))[0]
prediction = np.argmax(prediction)
print(prediction_list[prediction])


#cv2.putText(blackboard, "prediction" + prediction_list[prediction], (20,450), cv2.FONT_HERSHEY_SIMPLEX,
#           0.8, (255,255,255), 2)

#while time.time() > start_time + 21 and time.time() < start_time + 25:
#    cv2.imshow("blackboard", blackboard)


#    if cv2.waitKey(1) & 0xFF == ord("q"):
#        break
    
points = deque(maxlen = 1000)
blackboard = np.zeros((512,512), np.uint8)

cv2.destroyAllWindows()
cap.release()

l
